In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [44]:
# URL da página principal da DGES com informação sobre o acesso ao ensino superior
url="https://www.dges.gov.pt/pt/pagina/regime-geral-ensino-superior-publico-concurso-nacional-de-acesso"

# Headers para simular um browser e evitar bloqueios
headers = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

try:
    # Faz o pedido HTTP e verifica se houve erros
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Cria objeto BeautifulSoup para fazer parse do HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
except requests.RequestException as e:
    print(f"Error fetching the page: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

# Encontra todos os links na página
links = soup.find_all('a')

# Filtra apenas os links que contêm "fase1" ou "1f" ou "f1" (primeira fase)
fase1_urls = []
for link in links:
    href = link.get('href')
    if href and ('fase1' in href.lower() or '1f' in href.lower() or 'f1' in href.lower()):
        fase1_urls.append(href)

# Corrige URLs que não começam com http/https
corrected_urls=[]
for url in fase1_urls:
    if not (url.startswith('https:') or url.startswith('http:')) :
        url=f'https:{url}'
        corrected_urls.append(url)
    else:
        corrected_urls.append(url)

# Limita a 20 URLs para processamento
corrected_urls=corrected_urls[0:21]

corrected_urls

['https://wwwcdn.dges.gov.pt/sites/default/files/fase1_24.xlsx',
 'https://wwwcdn.dges.gov.pt/sites/default/files/fase1_a23_0.xlsx',
 'https://wwwcdn.dges.gov.pt/sites/default/files/cna22_1f_resultados.xls',
 'https://wwwcdn.dges.gov.pt/sites/default/files/cna21_1f_resultados.xls',
 'https://wwwcdn.dges.gov.pt/sites/default/files/cna20_1f_resultados.xls',
 'https://wwwcdn.dges.gov.pt/sites/default/files/site_cna19_1f_resultados.xls',
 'https://www.dges.gov.pt/sites/default/files/cna18_1f_resultados.ods',
 'https://www.dges.gov.pt/sites/default/files/cna17_1f_resultados.ods',
 'https://www.dges.gov.pt/sites/default/files/cna16_1f_resultados.ods',
 'https://www.dges.gov.pt/sites/default/files/ficheiros_excel_acesso/cna15_1f_resultados.xls',
 'https://www.dges.gov.pt/sites/default/files/ficheiros_excel_acesso/cna14_1f_resultados.xls',
 'https://www.dges.gov.pt/sites/default/files/ficheiros_excel_acesso/cna_13_1f_resultados_0.xls',
 'https://www.dges.gov.pt/sites/default/files/ficheiros_ex

## Limpeza de cada um dos requests individualmente

Apesar de serem todos relativamente parecidos, cada um tem ligeiras diferenças que impedem a criação de um script automatizado para todos. Os passos aplicados são os seguintes:
1. Leitura do excel
2. Promoção do cabeçalho ( Ajusta o cabeçalho - a linha 3 contém os nomes das colunas)
3. Apagar linhas e colunas vazias 
4. Renomear colunas
5. Converter colunas numéricas
6. Adicionar colunas de fase e ano

In [45]:
# Lê o primeiro excel da lista de URLs (dados de 2024)
df24=pd.read_excel(corrected_urls[0])

# Ajusta o cabeçalho - a linha 3 contém os nomes das colunas
df24.columns = df24.iloc[3]
# Remove as primeiras linhas que não são dados
df24 = df24.iloc[4:]
# Remove a primeira coluna que contém apenas índices
df24 = df24.iloc[:, 1:]
# Remove as últimas 3 linhas que contêm informações irrelevantes
df24 = df24.iloc[:-3]
df24

# Define o mapeamento para renomear as colunas para um formato padronizado
column_mapping = {
    'Código Instit.': 'codigo_instituicao',
    'Nome da Instituição': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Seleciona apenas as colunas que queremos e aplica os novos nomes
df24 = df24[column_mapping.keys()]
df24 = df24.rename(columns=column_mapping)

# Converte colunas que devem ser numéricas
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df24.columns:
        df24[col] = pd.to_numeric(df24[col], errors='coerce')

# Adiciona colunas de metadados - ano e fase do concurso
df24['ano']=2024
df24['fase']=1

df24



3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0140,Universidade dos Açores - Faculdade de Ciência...,8031,Ciências Farmacêuticas (Preparatórios),13,2,152.5,11,2024,1
5,0140,Universidade dos Açores - Faculdade de Ciência...,8086,Medicina Veterinária (Preparatórios),21,21,164.1,0,2024,1
6,0140,Universidade dos Açores - Faculdade de Ciência...,9022,Ciências Agrárias,16,5,NaN,11,2024,1
7,0140,Universidade dos Açores - Faculdade de Ciência...,L344,Guias de Natureza e Património,19,7,120.3,12,2024,1
8,0150,Universidade dos Açores - Faculdade de Ciência...,9135,Estudos Europeus,20,17,115.6,3,2024,1
...,...,...,...,...,...,...,...,...,...,...
1118,7230,Instituto Politécnico do Porto - Escola Superi...,L067,Fisiologia Clínica,48,48,145.5,0,2024,1
1119,7230,Instituto Politécnico do Porto - Escola Superi...,L068,Ciências Biomédicas Laboratoriais,60,60,166.0,0,2024,1
1120,7230,Instituto Politécnico do Porto - Escola Superi...,L101,Biotecnologia Medicinal,32,32,170.0,0,2024,1
1121,7230,Instituto Politécnico do Porto - Escola Superi...,L136,Osteopatia,21,21,143.8,0,2024,1


In [46]:
# Limpeza dos dados para o ano de 2023
# Lê o arquivo Excel com os dados de 2023
df23=pd.read_excel(corrected_urls[1])

# Ajusta o cabeçalho - a linha 3 contém os nomes das colunas
df23.columns = df23.iloc[3]
# Remove as primeiras linhas que não são dados
df23 = df23.iloc[4:]
# Remove a primeira coluna que contém apenas índices
df23 = df23.iloc[:, 1:]
# Remove as últimas 3 linhas que contêm informações irrelevantes
df23 = df23.iloc[:-3]
df23

# Define o mapeamento para renomear as colunas para um formato padronizado
column_mapping = {
    'Código Instit.': 'codigo_instituicao',
    'Nome da Instituição': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Seleciona apenas as colunas que queremos e aplica os novos nomes
df23 = df23[column_mapping.keys()]
df23 = df23.rename(columns=column_mapping)

# Converte colunas que devem ser numéricas
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df23.columns:
        df23[col] = pd.to_numeric(df23[col], errors='coerce')

# Adiciona colunas de metadados - ano e fase do concurso
df23['ano']=2023
df23['fase']=1

df23


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0140,Universidade dos Açores - Faculdade de Ciência...,8031,Ciências Farmacêuticas (Preparatórios),13,2,151.8,11,2023,1
5,0140,Universidade dos Açores - Faculdade de Ciência...,8086,Medicina Veterinária (Preparatórios),21,21,163.9,0,2023,1
6,0140,Universidade dos Açores - Faculdade de Ciência...,9022,Ciências Agrárias,16,6,128.4,10,2023,1
7,0140,Universidade dos Açores - Faculdade de Ciência...,L121,Natureza e Património,19,7,136.2,12,2023,1
8,0150,Universidade dos Açores - Faculdade de Ciência...,9135,Estudos Europeus,20,20,119.7,0,2023,1
...,...,...,...,...,...,...,...,...,...,...
1118,7230,Instituto Politécnico do Porto - Escola Superi...,L067,Fisiologia Clínica,48,48,153.0,0,2023,1
1119,7230,Instituto Politécnico do Porto - Escola Superi...,L068,Ciências Biomédicas Laboratoriais,60,60,164.5,0,2023,1
1120,7230,Instituto Politécnico do Porto - Escola Superi...,L101,Biotecnologia Medicinal,32,32,174.5,0,2023,1
1121,7230,Instituto Politécnico do Porto - Escola Superi...,L136,Osteopatia,21,21,139.8,0,2023,1


In [47]:
# Lê o arquivo Excel com os dados de 2022
df22=pd.read_excel(corrected_urls[2])

# Ajusta o cabeçalho - a linha 3 contém os nomes das colunas
df22.columns = df22.iloc[3]
# Remove as primeiras linhas que não são dados
df22 = df22.iloc[4:]
# Remove a primeira coluna que contém apenas índices
df22 = df22.iloc[:, 1:]
# Remove as últimas 3 linhas que contêm informações irrelevantes
df22 = df22.iloc[:-3]
df22

# Define o mapeamento para renomear as colunas para um formato padronizado
column_mapping = {
    'Código Instit.': 'codigo_instituicao',
    'Nome da Instituição': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Seleciona apenas as colunas que queremos e aplica os novos nomes
df22 = df22[column_mapping.keys()]
df22 = df22.rename(columns=column_mapping)

# Converte colunas que devem ser numéricas
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df22.columns:
        df22[col] = pd.to_numeric(df22[col], errors='coerce')

# Adiciona colunas de metadados - ano e fase do concurso
df22['ano']=2022
df22['fase']=1

df22


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0140,Universidade dos Açores - Faculdade de Ciência...,8031,Ciências Farmacêuticas (Preparatórios),16,10,130.3,6,2022,1
5,0140,Universidade dos Açores - Faculdade de Ciência...,8086,Medicina Veterinária (Preparatórios),15,15,167.9,0,2022,1
6,0140,Universidade dos Açores - Faculdade de Ciência...,9022,Ciências Agrárias,20,13,115.2,7,2022,1
7,0140,Universidade dos Açores - Faculdade de Ciência...,L121,Natureza e Património,25,4,115.0,20,2022,1
8,0150,Universidade dos Açores - Faculdade de Ciência...,9135,Estudos Europeus,20,18,109.5,2,2022,1
...,...,...,...,...,...,...,...,...,...,...
1102,7230,Instituto Politécnico do Porto - Escola Superi...,L066,Imagem Médica e Radioterapia,61,61,154.0,0,2022,1
1103,7230,Instituto Politécnico do Porto - Escola Superi...,L067,Fisiologia Clínica,48,48,159.0,0,2022,1
1104,7230,Instituto Politécnico do Porto - Escola Superi...,L068,Ciências Biomédicas Laboratoriais,61,61,173.5,0,2022,1
1105,7230,Instituto Politécnico do Porto - Escola Superi...,L101,Biotecnologia Medicinal,32,32,180.5,0,2022,1


In [48]:
# Lê o arquivo Excel com os dados de 2021
df21=pd.read_excel(corrected_urls[3])

# Ajusta o cabeçalho - a linha 3 contém os nomes das colunas
df21.columns = df21.iloc[3]
# Remove as primeiras linhas que não são dados
df21 = df21.iloc[4:]
# Remove a primeira coluna que contém apenas índices
df21 = df21.iloc[:, 1:]
# Remove as últimas 3 linhas que contêm informações irrelevantes
df21 = df21.iloc[:-3]

# Remove a primeira linha que contém informações desnecessárias
df21 = df21.iloc[1:]

# Define o mapeamento para renomear as colunas para um formato padronizado
column_mapping = {
    'Código Instit.': 'codigo_instituicao',
    'Nome da Instituição': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Seleciona apenas as colunas que queremos e aplica os novos nomes
df21 = df21[column_mapping.keys()]
df21 = df21.rename(columns=column_mapping)

# Converte colunas que devem ser numéricas
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df21.columns:
        df21[col] = pd.to_numeric(df21[col], errors='coerce')

# Adiciona colunas de metadados - ano e fase do concurso
df21['ano']=2021
df21['fase']=1

df21

3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
5,0140,Universidade dos Açores - Faculdade de Ciência...,8031,Ciências Farmacêuticas (Preparatórios),15,7,144.8,8,2021,1
6,0140,Universidade dos Açores - Faculdade de Ciência...,8086,Medicina Veterinária (Preparatórios),15,15,170.4,0,2021,1
7,0140,Universidade dos Açores - Faculdade de Ciência...,9022,Ciências Agrárias,20,10,NaN,10,2021,1
8,0140,Universidade dos Açores - Faculdade de Ciência...,L121,Natureza e Património,25,8,116.4,17,2021,1
9,0150,Universidade dos Açores - Faculdade de Ciência...,9181,História,22,22,135.0,0,2021,1
...,...,...,...,...,...,...,...,...,...,...
1069,7230,Instituto Politécnico do Porto - Escola Superi...,9890,Terapia da Fala,47,47,143.0,0,2021,1
1070,7230,Instituto Politécnico do Porto - Escola Superi...,L066,Imagem Médica e Radioterapia,55,55,159.0,0,2021,1
1071,7230,Instituto Politécnico do Porto - Escola Superi...,L067,Fisiologia Clínica,47,47,163.0,0,2021,1
1072,7230,Instituto Politécnico do Porto - Escola Superi...,L068,Ciências Biomédicas Laboratoriais,65,66,174.5,0,2021,1


In [49]:
# Lê o arquivo Excel com os dados de 2020
df20=pd.read_excel(corrected_urls[4])

# Define o cabeçalho usando a linha 3 que contém os nomes das colunas
df20.columns = df20.iloc[3]

# Remove as primeiras 4 linhas que não são dados
df20 = df20.iloc[4:]

# Remove a primeira coluna que contém apenas índices
df20 = df20.iloc[:, 1:]

# Remove as últimas 3 linhas que contêm informações irrelevantes
df20 = df20.iloc[:-3]

# Remove a primeira linha que contém informações desnecessárias
df20 = df20.iloc[1:]

# Define o mapeamento para renomear as colunas para um formato padronizado
column_mapping = {
    'Código Instit.': 'codigo_instituicao',
    'Nome da Instituição': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Seleciona apenas as colunas que queremos e aplica os novos nomes
df20 = df20[column_mapping.keys()]
df20 = df20.rename(columns=column_mapping)

# Converte colunas que devem ser numéricas
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df20.columns:
        df20[col] = pd.to_numeric(df20[col], errors='coerce')

# Adiciona colunas de metadados - ano e fase do concurso
df20['ano']=2020
df20['fase']=1

df20

3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
5,0140,Universidade dos Açores - Faculdade de Ciência...,8031,Ciências Farmacêuticas (Preparatórios),18,7,141.8,11,2020,1
6,0140,Universidade dos Açores - Faculdade de Ciência...,8086,Medicina Veterinária (Preparatórios),15,15,167.7,0,2020,1
7,0140,Universidade dos Açores - Faculdade de Ciência...,9022,Ciências Agrárias,20,11,114.3,9,2020,1
8,0140,Universidade dos Açores - Faculdade de Ciência...,L121,Natureza e Património,25,14,123.6,11,2020,1
9,0150,Universidade dos Açores - Faculdade de Ciência...,9181,História,21,21,138.5,0,2020,1
...,...,...,...,...,...,...,...,...,...,...
1064,7230,Instituto Politécnico do Porto - Escola Superi...,9890,Terapia da Fala,47,47,134.5,0,2020,1
1065,7230,Instituto Politécnico do Porto - Escola Superi...,L066,Imagem Médica e Radioterapia,62,63,150.0,0,2020,1
1066,7230,Instituto Politécnico do Porto - Escola Superi...,L067,Fisiologia Clínica,47,50,156.5,0,2020,1
1067,7230,Instituto Politécnico do Porto - Escola Superi...,L068,Ciências Biomédicas Laboratoriais,62,63,165.0,0,2020,1


In [50]:
# Lê o arquivo Excel com os dados de 2019
df19=pd.read_excel(corrected_urls[5])

# Define o cabeçalho usando a linha 3 que contém os nomes das colunas
df19.columns = df19.iloc[3]

# Remove as primeiras 4 linhas que não são dados
df19 = df19.iloc[4:]

# Remove a primeira coluna que contém apenas índices
df19 = df19.iloc[:, 1:]

# Remove as últimas 3 linhas que contêm informações irrelevantes
df19 = df19.iloc[:-3]

# Remove a primeira linha que contém informações desnecessárias
df19 = df19.iloc[1:]

# Define o mapeamento para renomear as colunas para um formato padronizado
column_mapping = {
    'Código Instit.': 'codigo_instituicao',
    'Nome da Instituição': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Seleciona apenas as colunas que queremos e aplica os novos nomes
df19 = df19[column_mapping.keys()]
df19 = df19.rename(columns=column_mapping)

# Converte colunas que devem ser numéricas
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df19.columns:
        df19[col] = pd.to_numeric(df19[col], errors='coerce')

# Adiciona colunas de metadados - ano e fase do concurso
df19['ano']=2019
df19['fase']=1

df19

3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
5,0140,Universidade dos Açores - Faculdade de Ciência...,8031,Ciências Farmacêuticas (Preparatórios),15,9,132.8,6,2019,1
6,0140,Universidade dos Açores - Faculdade de Ciência...,8086,Medicina Veterinária (Preparatórios),15,15,148.0,0,2019,1
7,0140,Universidade dos Açores - Faculdade de Ciência...,9022,Ciências Agrárias,20,2,NaN,18,2019,1
8,0140,Universidade dos Açores - Faculdade de Ciência...,L121,Natureza e Património,25,6,NaN,19,2019,1
9,0150,Universidade dos Açores - Faculdade de Ciência...,9181,História,20,16,118.5,4,2019,1
...,...,...,...,...,...,...,...,...,...,...
1060,7230,Instituto Politécnico do Porto - Escola Superi...,9890,Terapia da Fala,38,38,117.5,0,2019,1
1061,7230,Instituto Politécnico do Porto - Escola Superi...,L066,Imagem Médica e Radioterapia,55,55,128.0,0,2019,1
1062,7230,Instituto Politécnico do Porto - Escola Superi...,L067,Fisiologia Clínica,43,43,136.5,0,2019,1
1063,7230,Instituto Politécnico do Porto - Escola Superi...,L068,Ciências Biomédicas Laboratoriais,55,55,153.3,0,2019,1


In [51]:
# Lê o arquivo Excel com os dados de 2018
df18=pd.read_excel(corrected_urls[6])

# Define o cabeçalho usando a linha 3 que contém os nomes das colunas
df18.columns = df18.iloc[3]

# Remove as primeiras 4 linhas que não são dados
df18 = df18.iloc[4:]

# Remove a primeira coluna que contém apenas índices
df18 = df18.iloc[:, 1:]

# Remove as últimas 3 linhas que contêm informações irrelevantes
df18 = df18.iloc[:-3]

# Remove a primeira linha que contém informações desnecessárias
df18 = df18.iloc[1:]

# Define o mapeamento para renomear as colunas para um formato padronizado
column_mapping = {
    'Código Instit.': 'codigo_instituicao',
    'Nome da Instituição': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para2ª fase': 'vagas_sobrantes'
}

# Seleciona apenas as colunas que queremos e aplica os novos nomes
df18 = df18[column_mapping.keys()]
df18 = df18.rename(columns=column_mapping)

# Converte colunas que devem ser numéricas
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df18.columns:
        df18[col] = pd.to_numeric(df18[col], errors='coerce')

# Adiciona colunas de metadados - ano e fase do concurso
df18['ano']=2018
df18['fase']=1

df18


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
5,0140,Universidade dos Açores - Faculdade de Ciência...,8031,Ciências Farmacêuticas (Preparatórios),15,10,109.0,5,2018,1
6,0140,Universidade dos Açores - Faculdade de Ciência...,8086,Medicina Veterinária (Preparatórios),15,15,152.0,0,2018,1
7,0140,Universidade dos Açores - Faculdade de Ciência...,9022,Ciências Agrárias,20,1,NaN,19,2018,1
8,0140,Universidade dos Açores - Faculdade de Ciência...,L121,Natureza e Património,30,7,119.3,23,2018,1
9,0150,Universidade dos Açores - Faculdade de Ciência...,8082,Arquitetura (Preparatórios),15,3,NaN,12,2018,1
...,...,...,...,...,...,...,...,...,...,...
1067,7230,Instituto Politécnico do Porto - Escola Superi...,9890,Terapia da Fala,40,40,116.0,0,2018,1
1068,7230,Instituto Politécnico do Porto - Escola Superi...,L066,Imagem Médica e Radioterapia,55,55,133.0,0,2018,1
1069,7230,Instituto Politécnico do Porto - Escola Superi...,L067,Fisiologia Clínica,43,43,136.0,0,2018,1
1070,7230,Instituto Politécnico do Porto - Escola Superi...,L068,Ciências Biomédicas Laboratoriais,55,55,149.0,0,2018,1


In [52]:
# Lê o arquivo Excel com os dados de 2017
df17=pd.read_excel(corrected_urls[7])

# Define o cabeçalho usando a linha 3 que contém os nomes das colunas
df17.columns = df17.iloc[3]

# Remove as primeiras 4 linhas que não são dados
df17 = df17.iloc[4:]

# Remove a primeira coluna que contém apenas índices
df17 = df17.iloc[:, 1:]

# Remove as últimas 3 linhas que contêm informações irrelevantes
df17 = df17.iloc[:-3]

# Remove a primeira linha que contém informações desnecessárias
df17 = df17.iloc[1:]

# Define o mapeamento para renomear as colunas para um formato padronizado
column_mapping = {
    'Código Instituição': 'codigo_instituicao',
    'Instituição de ensino superior': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (contingente geral)': 'nota_ultimo_colocado',
    'Sobras para2ª fase': 'vagas_sobrantes'
}

# Mostra o dataframe antes das transformações
df17

# Seleciona apenas as colunas que queremos e aplica os novos nomes
df17 = df17[column_mapping.keys()]
df17 = df17.rename(columns=column_mapping)

# Converte colunas que devem ser numéricas
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df17.columns:
        df17[col] = pd.to_numeric(df17[col], errors='coerce')

# Adiciona colunas de metadados - ano e fase do concurso
df17['ano']=2017
df17['fase']=1

# Mostra o dataframe após todas as transformações
df17


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
5,0140,Universidade dos Açores - Faculdade de Ciência...,8031,Ciências Farmacêuticas (Preparatórios),15,9,119.8,6,2017,1
6,0140,Universidade dos Açores - Faculdade de Ciência...,8086,Medicina Veterinária (Preparatórios),15,15,153.8,0,2017,1
7,0140,Universidade dos Açores - Faculdade de Ciência...,9022,Ciências Agrárias,20,8,120.4,12,2017,1
8,0140,Universidade dos Açores - Faculdade de Ciência...,L121,Natureza e Património,30,12,128.5,18,2017,1
9,0150,Universidade dos Açores - Faculdade de Ciência...,8082,Arquitetura (Preparatórios),15,1,NaN,14,2017,1
...,...,...,...,...,...,...,...,...,...,...
1061,7230,Instituto Politécnico do Porto - Escola Superi...,9890,Terapia da Fala,40,40,114.5,0,2017,1
1062,7230,Instituto Politécnico do Porto - Escola Superi...,L066,Imagem Médica e Radioterapia,60,61,131.0,0,2017,1
1063,7230,Instituto Politécnico do Porto - Escola Superi...,L067,Fisiologia Clínica,45,45,134.5,0,2017,1
1064,7230,Instituto Politécnico do Porto - Escola Superi...,L068,Ciências Biomédicas Laboratoriais,57,57,145.3,0,2017,1


In [53]:
df16=pd.read_excel(corrected_urls[8])

# Promote row 3 to header 
df16.columns = df16.iloc[1]
df16 = df16.iloc[4:]

#delete last 3 rows
df16 = df16.iloc[:-2]
# Remove as últimas 2 linhas que contêm informações irrelevantes
df16 = df16.iloc[:-2]

#delete first row 
df16 = df16.iloc[1:]

###rename columns
column_mapping = {
    'Código da instituição': 'codigo_instituicao',
    'Nome da instituição': 'instituicao',
    'Código do curso': 'codigo_curso', 
    'Nome do curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Estudantes colocados': 'colocados',
    'Nota de candidatura do último colocado pelo contingente geral': 'nota_ultimo_colocado',
    'Vagas sobrantes': 'vagas_sobrantes'
}

#apply mapping
df16 = df16[column_mapping.keys()]
df16 = df16.rename(columns=column_mapping)

#convert numeric columns
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df16.columns:
        df16[col] = pd.to_numeric(df16[col], errors='coerce')


df16['ano']=2016
df16['fase']=1

df16


1,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
5,0140,Universidade dos Açores - Faculdade de Ciência...,9022,Ciências Agrárias,20,6,NaN,14,2016,1
6,0140,Universidade dos Açores - Faculdade de Ciência...,L121,Natureza e Património,30,13,125.1,17,2016,1
7,0150,Universidade dos Açores - Faculdade de Ciência...,8082,Arquitetura (Preparatórios),20,3,136.5,17,2016,1
8,0150,Universidade dos Açores - Faculdade de Ciência...,9181,História,20,4,108.7,16,2016,1
9,0150,Universidade dos Açores - Faculdade de Ciência...,9219,Psicologia,28,28,127.8,0,2016,1
...,...,...,...,...,...,...,...,...,...,...
1056,7230,Instituto Politécnico do Porto - Escola Superi...,9861,Saúde Ambiental,41,30,104.0,11,2016,1
1057,7230,Instituto Politécnico do Porto - Escola Superi...,9890,Terapia da Fala,40,40,118.5,0,2016,1
1058,7230,Instituto Politécnico do Porto - Escola Superi...,L066,Imagem Médica e Radioterapia,60,60,129.0,0,2016,1
1059,7230,Instituto Politécnico do Porto - Escola Superi...,L067,Fisiologia Clínica,45,45,131.5,0,2016,1


In [54]:
# Ler o ficheiro Excel para 2015
df15=pd.read_excel(corrected_urls[9])

# Definir a linha 3 como cabeçalho da tabela
df15.columns = df15.iloc[3]
df15 = df15.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df15 = df15.iloc[:-2]

# Remover a primeira linha que contém informações desnecessárias
df15 = df15.iloc[1:]

# Mapear os nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Código Instit.': 'codigo_instituicao',
    'Instituição': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Vagas sobrantes': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df15.columns)

# Selecionar apenas as colunas necessárias e renomeá-las
df15 = df15[column_mapping.keys()]
df15 = df15.rename(columns=column_mapping)

# Converter colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df15.columns:
        df15[col] = pd.to_numeric(df15[col], errors='coerce')

# Adicionar colunas de ano e fase
df15['ano']=2015
df15['fase']=1

df15

Index(['Código Instit.', 'Código Curso', 'Instituição', 'Curso', 'Grau',
       'Vagas Iniciais', 'Colocados', 'Nota do últ. colocado (cont. geral)',
       'Vagas sobrantes'],
      dtype='object', name=3)


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
5,0110,Universidade dos Açores - Angra do Heroísmo,8031,Ciências Farmacêuticas (Preparatórios),15,11,129.3,4,2015,1
6,0110,Universidade dos Açores - Angra do Heroísmo,8086,Medicina Veterinária (Preparatórios),13,13,145.5,0,2015,1
7,0110,Universidade dos Açores - Angra do Heroísmo,9022,Ciências Agrárias,20,4,119.0,16,2015,1
8,0110,Universidade dos Açores - Angra do Heroísmo,9382,Guias da Natureza,20,14,115.3,6,2015,1
9,0110,Universidade dos Açores - Angra do Heroísmo,9934,Energias Renováveis,20,2,NaN,18,2015,1
...,...,...,...,...,...,...,...,...,...,...
1048,7230,Instituto Politécnico do Porto - Escola Superi...,9890,Terapia da Fala,40,40,122.5,0,2015,1
1049,7230,Instituto Politécnico do Porto - Escola Superi...,L066,Imagem Médica e Radioterapia,60,60,134.8,0,2015,1
1050,7230,Instituto Politécnico do Porto - Escola Superi...,L067,Fisiologia Clínica,45,45,141.5,0,2015,1
1051,7230,Instituto Politécnico do Porto - Escola Superi...,L068,Ciências Biomédicas Laboratoriais,57,57,143.0,0,2015,1


In [55]:
# Ler os dados do ficheiro Excel para 2014
df14=pd.read_excel(corrected_urls[10])

# Definir a primeira linha como cabeçalho da tabela
df14.columns = df14.iloc[0]
# Começar a ler os dados a partir da linha 4 (índice 4)
df14 = df14.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df14 = df14.iloc[:-2]

# Definir o mapeamento dos nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Código da instituição': 'codigo_instituicao',
    'Nome da instituição': 'instituicao',
    'Código do curso': 'codigo_curso', 
    'Nome do curso': 'curso',
    'Vagas iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota de candidatura do último colocado pelo contingente geral': 'nota_ultimo_colocado',
    'Vagas sobrantes': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df14.columns)

# Selecionar apenas as colunas necessárias e renomeá-las usando o mapeamento
df14 = df14[column_mapping.keys()]
df14 = df14.rename(columns=column_mapping)

# Converter as colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df14.columns:
        df14[col] = pd.to_numeric(df14[col], errors='coerce')

# Adicionar colunas de ano e fase
df14['ano']=2014
df14['fase']=1

# Mostrar o dataframe resultante
df14

Index(['Código da instituição', 'Código do curso', 'Nome da instituição',
       'Nome do curso', 'Grau', 'Vagas iniciais', 'Colocados ',
       'Nota de candidatura do último colocado pelo contingente geral',
       'Vagas sobrantes'],
      dtype='object', name=0)


,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0110,Universidade dos Açores - Angra do Heroísmo,8086,Medicina Veterinária (Preparatórios),13,13,144.3,0,2014,1
5,0110,Universidade dos Açores - Angra do Heroísmo,9022,Ciências Agrárias,20,4,NaN,16,2014,1
6,0110,Universidade dos Açores - Angra do Heroísmo,9099,Engenharia do Ambiente,20,0,NaN,20,2014,1
7,0110,Universidade dos Açores - Angra do Heroísmo,9382,Guias da Natureza,20,4,133.8,16,2014,1
8,0110,Universidade dos Açores - Angra do Heroísmo,9934,Energias Renováveis,20,5,120.7,15,2014,1
...,...,...,...,...,...,...,...,...,...,...
1064,7230,Instituto Politécnico do Porto - Escola Superi...,9504,Fisioterapia,56,57,158.5,0,2014,1
1065,7230,Instituto Politécnico do Porto - Escola Superi...,9505,Radiologia,42,42,135.5,0,2014,1
1066,7230,Instituto Politécnico do Porto - Escola Superi...,9549,Farmácia,56,56,141.5,0,2014,1
1067,7230,Instituto Politécnico do Porto - Escola Superi...,9861,Saúde Ambiental,41,41,116.5,0,2014,1


In [56]:
# Ler os dados do ficheiro Excel para 2013
df13=pd.read_excel(corrected_urls[11])

# Definir a primeira linha como cabeçalho da tabela
df13.columns = df13.iloc[0]
# Começar a ler os dados a partir da linha 4 (índice 4)
df13 = df13.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df13 = df13.iloc[:-2]

# Definir o mapeamento dos nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Código da instituição': 'codigo_instituicao',
    'Nome da instituição': 'instituicao',
    'Código do curso': 'codigo_curso', 
    'Nome do curso': 'curso',
    'Vagas iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota de candidatura do último colocado pelo contingente geral': 'nota_ultimo_colocado',
    'Vagas sobrantes': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df13.columns)

# Selecionar apenas as colunas necessárias e renomeá-las usando o mapeamento
df13 = df13[column_mapping.keys()]
df13 = df13.rename(columns=column_mapping)

# Converter as colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df13.columns:
        df13[col] = pd.to_numeric(df13[col], errors='coerce')

# Adicionar colunas de ano e fase
df13['ano']=2013
df13['fase']=1

# Mostrar o dataframe resultante
df13

Index(['Código da instituição', 'Código do curso', 'Nome da instituição',
       'Nome do curso', 'Grau', 'Vagas iniciais', 'Colocados ',
       'Nota de candidatura do último colocado pelo contingente geral',
       'Vagas sobrantes'],
      dtype='object', name=0)


,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0110,Universidade dos Açores - Angra do Heroísmo,8086,Medicina Veterinária (Preparatórios),13,11,132.5,2,2013,1
5,0110,Universidade dos Açores - Angra do Heroísmo,9022,Ciências Agrárias,20,7,136.9,13,2013,1
6,0110,Universidade dos Açores - Angra do Heroísmo,9103,Engenharia e Gestão do Ambiente,20,0,NaN,20,2013,1
7,0110,Universidade dos Açores - Angra do Heroísmo,9382,Guias da Natureza,20,8,163.4,12,2013,1
8,0110,Universidade dos Açores - Angra do Heroísmo,9934,Energias Renováveis,20,6,133.5,14,2013,1
...,...,...,...,...,...,...,...,...,...,...
1084,7230,Instituto Politécnico do Porto - Escola Superi...,9504,Fisioterapia,56,57,153.5,0,2013,1
1085,7230,Instituto Politécnico do Porto - Escola Superi...,9505,Radiologia,42,42,133.0,0,2013,1
1086,7230,Instituto Politécnico do Porto - Escola Superi...,9549,Farmácia,56,56,133.5,0,2013,1
1087,7230,Instituto Politécnico do Porto - Escola Superi...,9861,Saúde Ambiental,41,41,117.5,0,2013,1


In [57]:
# Ler os dados do ficheiro Excel para 2012
df12=pd.read_excel(corrected_urls[12])

# Definir a primeira linha como cabeçalho da tabela
df12.columns = df12.iloc[0]
# Começar a ler os dados a partir da linha 4 (índice 4)
df12 = df12.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df12 = df12.iloc[:-2]

# Definir o mapeamento dos nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Código da instituição': 'codigo_instituicao',
    'Nome da instituição': 'instituicao',
    'Código do curso': 'codigo_curso', 
    'Nome do curso': 'curso',
    'Vagas iniciais': 'vagas_iniciais',
    'Colocados': 'colocados',
    'Nota de candidatura do último colocado pelo contingente geral': 'nota_ultimo_colocado',
    'Vagas sobrantes': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df12.columns)

# Selecionar apenas as colunas necessárias e renomeá-las usando o mapeamento
df12 = df12[column_mapping.keys()]
df12 = df12.rename(columns=column_mapping)

# Converter as colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df12.columns:
        df12[col] = pd.to_numeric(df12[col], errors='coerce')

# Adicionar colunas de ano e fase
df12['ano']=2012
df12['fase']=1

# Mostrar o dataframe resultante
df12

Index(['Código da instituição', 'Código do curso', 'Nome da instituição',
       'Nome do curso', 'Grau', 'Vagas iniciais', 'Colocados',
       'Nota de candidatura do último colocado pelo contingente geral',
       'Vagas sobrantes'],
      dtype='object', name=0)


,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0110,Universidade dos Açores - Angra do Heroísmo,8086,Medicina Veterinária (Preparatórios),13,13,156.3,0,2012,1
5,0110,Universidade dos Açores - Angra do Heroísmo,9022,Ciências Agrárias,10,7,117.2,3,2012,1
6,0110,Universidade dos Açores - Angra do Heroísmo,9103,Engenharia e Gestão do Ambiente,20,0,NaN,20,2012,1
7,0110,Universidade dos Açores - Angra do Heroísmo,9382,Guias da Natureza,20,7,142.8,13,2012,1
8,0110,Universidade dos Açores - Angra do Heroísmo,9934,Energias Renováveis,20,17,114.3,3,2012,1
...,...,...,...,...,...,...,...,...,...,...
1119,7230,Instituto Politécnico do Porto - Escola Superi...,9504,Fisioterapia,60,60,157.5,0,2012,1
1120,7230,Instituto Politécnico do Porto - Escola Superi...,9505,Radiologia,45,46,145.0,0,2012,1
1121,7230,Instituto Politécnico do Porto - Escola Superi...,9549,Farmácia,60,60,148.5,0,2012,1
1122,7230,Instituto Politécnico do Porto - Escola Superi...,9861,Saúde Ambiental,45,45,132.5,0,2012,1


In [58]:
# Ler os dados do ficheiro Excel para 2011
df11=pd.read_excel(corrected_urls[13])

# Definir a linha 3 como cabeçalho da tabela
df11.columns = df11.iloc[3]
# Começar a ler os dados a partir da linha 4
df11 = df11.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df11 = df11.iloc[:-2]
# Remover a primeira coluna que não é necessária
df11 = df11.iloc[:, 1:]

# Definir o mapeamento dos nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Código Instit.': 'codigo_instituicao',
    'Nome da Instituição': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df11.columns)

# Selecionar apenas as colunas necessárias e renomeá-las usando o mapeamento
df11 = df11[column_mapping.keys()]
df11 = df11.rename(columns=column_mapping)

# Converter as colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df11.columns:
        df11[col] = pd.to_numeric(df11[col], errors='coerce')

# Adicionar colunas de ano e fase
df11['ano']=2011
df11['fase']=1

# Mostrar o dataframe resultante
df11

Index(['Código Instit.', 'Código Curso', 'Nome da Instituição',
       'Nome do Curso', 'Grau', 'Vagas Iniciais', 'Colocados ',
       'Colocados (desemp.)', 'Colocados (sem class. final)',
       'Nota do últ. colocado (cont. geral)', 'Sobras para\n2ª fase'],
      dtype='object', name=3)


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0110,Universidade dos Açores - Angra do Heroísmo,8031,Ciências Farmacêuticas (Preparatórios),15,15,154.0,0,2011,1
5,0110,Universidade dos Açores - Angra do Heroísmo,8085,Ciências da Nutrição (Preparatórios),20,9,142.8,11,2011,1
6,0110,Universidade dos Açores - Angra do Heroísmo,8086,Medicina Veterinária (Preparatórios),12,12,156.5,0,2011,1
7,0110,Universidade dos Açores - Angra do Heroísmo,9022,Ciências Agrárias,10,2,NaN,8,2011,1
8,0110,Universidade dos Açores - Angra do Heroísmo,9103,Engenharia e Gestão do Ambiente,20,4,142.2,16,2011,1
...,...,...,...,...,...,...,...,...,...,...
1151,7230,Instituto Politécnico do Porto - Escola Superi...,9504,Fisioterapia,60,60,167.0,0,2011,1
1152,7230,Instituto Politécnico do Porto - Escola Superi...,9505,Radiologia,45,45,150.0,0,2011,1
1153,7230,Instituto Politécnico do Porto - Escola Superi...,9549,Farmácia,60,60,157.5,0,2011,1
1154,7230,Instituto Politécnico do Porto - Escola Superi...,9861,Saúde Ambiental,45,45,132.5,0,2011,1


In [59]:
# Ler os dados do ficheiro Excel para 2010
df10=pd.read_excel(corrected_urls[14])

# Definir a linha 3 como cabeçalho da tabela
df10.columns = df10.iloc[3]
# Começar a ler os dados a partir da linha 4
df10 = df10.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df10 = df10.iloc[:-2]
# Remover a primeira coluna que não é necessária
df10 = df10.iloc[:, 1:]

# Definir o mapeamento dos nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Código Estab.': 'codigo_instituicao',
    'Nome do Estabelecimento': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df10.columns)

# Selecionar apenas as colunas necessárias e renomeá-las usando o mapeamento
df10 = df10[column_mapping.keys()]
df10 = df10.rename(columns=column_mapping)

# Converter as colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df10.columns:
        df10[col] = pd.to_numeric(df10[col], errors='coerce')

# Adicionar colunas de ano e fase
df10['ano']=2010
df10['fase']=1

# Mostrar o dataframe resultante
df10

Index(['Código Estab.', 'Código Curso', 'Nome do Estabelecimento',
       'Nome do Curso', 'Grau', 'Vagas Iniciais', 'Colocados ',
       'Colocados (vaga adic.)', 'Nota do últ. colocado (cont. geral)',
       'Sobras para\n2ª fase'],
      dtype='object', name=3)


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0110,Universidade dos Açores - Angra do Heroísmo,8031,Ciências Farmacêuticas (Preparatórios),15,15,155.8,0,2010,1
5,0110,Universidade dos Açores - Angra do Heroísmo,8085,Ciências da Nutrição (Preparatórios),20,5,135.0,15,2010,1
6,0110,Universidade dos Açores - Angra do Heroísmo,8086,Medicina Veterinária (Preparatórios),12,12,155.8,0,2010,1
7,0110,Universidade dos Açores - Angra do Heroísmo,9022,Ciências Agrárias,10,7,124.6,3,2010,1
8,0110,Universidade dos Açores - Angra do Heroísmo,9103,Engenharia e Gestão do Ambiente,20,9,111.3,11,2010,1
...,...,...,...,...,...,...,...,...,...,...
1151,7230,Instituto Politécnico do Porto - Escola Superi...,9504,Fisioterapia,60,60,171.5,0,2010,1
1152,7230,Instituto Politécnico do Porto - Escola Superi...,9505,Radiologia,45,45,155.5,0,2010,1
1153,7230,Instituto Politécnico do Porto - Escola Superi...,9549,Farmácia,60,60,162.0,0,2010,1
1154,7230,Instituto Politécnico do Porto - Escola Superi...,9861,Saúde Ambiental,45,45,129.0,0,2010,1


In [60]:
# Ler os dados do ficheiro Excel para 2009
df9=pd.read_excel(corrected_urls[15])

# Definir a linha 3 como cabeçalho da tabela
df9.columns = df9.iloc[3]
# Começar a ler os dados a partir da linha 4
df9 = df9.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df9 = df9.iloc[:-2]
# Remover a primeira coluna que não é necessária
df9 = df9.iloc[:, 1:]

# Definir o mapeamento dos nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Código Estab.': 'codigo_instituicao',
    'Nome do Estabelecimento': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df9.columns)

# Selecionar apenas as colunas necessárias e renomeá-las usando o mapeamento
df9 = df9[column_mapping.keys()]
df9 = df9.rename(columns=column_mapping)

# Converter as colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df9.columns:
        df9[col] = pd.to_numeric(df9[col], errors='coerce')

# Adicionar colunas de ano e fase
df9['ano']=2009
df9['fase']=1

# Mostrar o dataframe resultante
df9

Index(['Código Estab.', 'Código Curso', 'Nome do Estabelecimento',
       'Nome do Curso', 'Grau', 'Vagas Iniciais', 'Colocados ',
       'Colocados (vaga adic.)', 'Nota do últ. colocado (cont. geral)',
       'Sobras para\n2ª fase'],
      dtype='object', name=3)


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0110,Universidade dos Açores - Angra do Heroísmo,8031,Ciências Farmacêuticas (Preparatórios),15,15,155.5,0,2009,1
5,0110,Universidade dos Açores - Angra do Heroísmo,8085,Ciências da Nutrição (Preparatórios),20,20,128.8,0,2009,1
6,0110,Universidade dos Açores - Angra do Heroísmo,8086,Medicina Veterinária (Preparatórios),12,12,154.8,0,2009,1
7,0110,Universidade dos Açores - Angra do Heroísmo,9022,Ciências Agrárias,10,1,132.7,9,2009,1
8,0110,Universidade dos Açores - Angra do Heroísmo,9103,Engenharia e Gestão do Ambiente,20,12,105.7,8,2009,1
...,...,...,...,...,...,...,...,...,...,...
1098,7230,Instituto Politécnico do Porto - Escola Superi...,9504,Fisioterapia,60,60,171.0,0,2009,1
1099,7230,Instituto Politécnico do Porto - Escola Superi...,9505,Radiologia,45,45,152.0,0,2009,1
1100,7230,Instituto Politécnico do Porto - Escola Superi...,9549,Farmácia,60,60,158.0,0,2009,1
1101,7230,Instituto Politécnico do Porto - Escola Superi...,9861,Saúde Ambiental,45,45,134.5,0,2009,1


In [61]:
# Ler os dados do ficheiro Excel para 2008
df8=pd.read_excel(corrected_urls[16])

# Definir a linha 3 como cabeçalho da tabela
df8.columns = df8.iloc[3]
# Começar a ler os dados a partir da linha 4
df8 = df8.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df8 = df8.iloc[:-2]
# Remover a primeira coluna que não é necessária
df8 = df8.iloc[:, 1:]

# Definir o mapeamento dos nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Código Estab.': 'codigo_instituicao',
    'Nome do Estabelecimento': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df8.columns)

# Selecionar apenas as colunas necessárias e renomeá-las usando o mapeamento
df8 = df8[column_mapping.keys()]
df8 = df8.rename(columns=column_mapping)

# Converter as colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df8.columns:
        df8[col] = pd.to_numeric(df8[col], errors='coerce')

# Adicionar colunas de ano e fase
df8['ano']=2008
df8['fase']=1

# Mostrar o dataframe resultante
df8

Index(['Código Estab.', 'Código Curso', 'Nome do Estabelecimento',
       'Nome do Curso', 'Grau', 'Vagas Iniciais', 'Colocados ',
       'Colocados (vaga adic.)', 'Nota do últ. colocado (cont. geral)',
       'Sobras para\n2ª fase'],
      dtype='object', name=3)


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0110,Universidade dos Açores - Angra do Heroísmo,8031,Ciências Farmacêuticas (Preparatórios),15,15,150.7,0,2008,1
5,0110,Universidade dos Açores - Angra do Heroísmo,8085,Ciências da Nutrição (Preparatórios),20,20,116.0,0,2008,1
6,0110,Universidade dos Açores - Angra do Heroísmo,8086,Medicina Veterinária (Preparatórios),12,12,153.0,0,2008,1
7,0110,Universidade dos Açores - Angra do Heroísmo,9022,Ciências Agrárias,10,9,106.7,1,2008,1
8,0110,Universidade dos Açores - Angra do Heroísmo,9103,Engenharia e Gestão do Ambiente,20,20,118.6,0,2008,1
...,...,...,...,...,...,...,...,...,...,...
1067,7230,Instituto Politécnico do Porto - Escola Superi...,1699,Radiologia,45,45,166.5,0,2008,1
1068,7230,Instituto Politécnico do Porto - Escola Superi...,1701,Radioterapia,20,20,167.0,0,2008,1
1069,7230,Instituto Politécnico do Porto - Escola Superi...,1728,Saúde Ambiental,45,45,135.5,0,2008,1
1070,7230,Instituto Politécnico do Porto - Escola Superi...,1775,Terapêutica da Fala,30,30,145.5,0,2008,1


In [62]:
# Ler os dados do ficheiro Excel para 2007
df7=pd.read_excel(corrected_urls[17])

# Definir a linha 2 como cabeçalho da tabela e começar a ler os dados a partir da linha 4
df7.columns = df7.iloc[2]
df7 = df7.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df7 = df7.iloc[:-2]
# Remover a primeira coluna que não é necessária
df7 = df7.iloc[:, 1:]

# Definir o mapeamento dos nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Cód. Estab.': 'codigo_instituicao',
    'Nome do Estabelecimento': 'instituicao',
    'Cód. Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas 2007': 'vagas_iniciais',
    'Colocados 2007': 'colocados',
    'Nota Últ. colocado': 'nota_ultimo_colocado',
    'Sobras p/ 2ª fase 2007': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df7.columns)

# Selecionar apenas as colunas necessárias e renomeá-las usando o mapeamento
df7 = df7[column_mapping.keys()]
df7 = df7.rename(columns=column_mapping)

# Converter as colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df7.columns:
        df7[col] = pd.to_numeric(df7[col], errors='coerce')

# Adicionar colunas de ano e fase
df7['ano']=2007
df7['fase']=1

# Mostrar o dataframe resultante
df7.dropna(inplace=True) ##cursos com vags iniciais nulas foram extinguidos

Index(['Cód. Estab.', 'Cód. Curso', 'Nome do Estabelecimento', 'Nome do Curso',
       'Grau', 'Vagas 2007', 'Candidatos\n1ºop 2007', 'Colocados 2007',
       'Adicionais 2007', 'Sobras p/ 2ª fase 2007', 'Nota Últ. colocado'],
      dtype='object', name=2)


In [63]:
# Ler os dados do ficheiro Excel para 2006
df6=pd.read_excel(corrected_urls[18])

# Definir a linha 3 como cabeçalho da tabela e começar a ler os dados a partir da linha 4
df6.columns = df6.iloc[3]
df6 = df6.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df6 = df6.iloc[:-2]
# Remover a primeira coluna que não é necessária
df6 = df6.iloc[:, 1:]

# Definir o mapeamento dos nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Código Estab.': 'codigo_instituicao',
    'Nome do Estabelecimento': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df6.columns)

# Selecionar apenas as colunas necessárias e renomeá-las usando o mapeamento
df6 = df6[column_mapping.keys()]
df6 = df6.rename(columns=column_mapping)

# Converter as colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df6.columns:
        df6[col] = pd.to_numeric(df6[col], errors='coerce')

# Adicionar colunas de ano e fase
df6['ano']=2006
df6['fase']=1

# Mostrar o dataframe resultante
df6

Index(['Código Estab.', 'Código Curso', 'Nome do Estabelecimento',
       'Nome do Curso', 'Grau', 'Vagas Iniciais', 'Colocados ',
       'Colocados (vaga adic.)', 'Nota do últ. colocado (cont. geral)',
       'Sobras para\n2ª fase'],
      dtype='object', name=3)


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0110,Universidade dos Açores - Angra do Heroísmo,0625,Educação de Infância,20,12,121.599998,8,2006,1
5,0110,Universidade dos Açores - Angra do Heroísmo,1131,Ciências da Nutrição (Preparatórios),20,16,120.500000,4,2006,1
6,0110,Universidade dos Açores - Angra do Heroísmo,1603,Medicina Veterinária (Preparatórios),12,12,154.500000,0,2006,1
7,0110,Universidade dos Açores - Angra do Heroísmo,9022,Ciências Agrárias,10,9,112.300003,1,2006,1
8,0110,Universidade dos Açores - Angra do Heroísmo,9103,Engenharia e Gestão do Ambiente,20,6,NaN,14,2006,1
...,...,...,...,...,...,...,...,...,...,...
990,7230,Instituto Politécnico do Porto - Escola Superi...,1699,Radiologia,45,45,149.500000,0,2006,1
991,7230,Instituto Politécnico do Porto - Escola Superi...,1701,Radioterapia,20,20,146.500000,0,2006,1
992,7230,Instituto Politécnico do Porto - Escola Superi...,1728,Saúde Ambiental,45,45,127.000000,0,2006,1
993,7230,Instituto Politécnico do Porto - Escola Superi...,1775,Terapêutica da Fala,30,30,160.500000,0,2006,1


In [64]:
# Ler os dados do ficheiro Excel para 2005 (2ª fonte)
df5=pd.read_excel(corrected_urls[19])

# Definir a linha 3 como cabeçalho da tabela e começar a ler os dados a partir da linha 4
df5.columns = df5.iloc[3]
df5 = df5.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df5 = df5.iloc[:-2]
# Remover a primeira coluna que não é necessária
df5 = df5.iloc[:, 1:]

# Definir o mapeamento dos nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Código Estab.': 'codigo_instituicao',
    'Nome do Estabelecimento': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df5.columns)

# Selecionar apenas as colunas necessárias e renomeá-las usando o mapeamento
df5 = df5[column_mapping.keys()]
df5 = df5.rename(columns=column_mapping)

# Converter as colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df5.columns:
        df5[col] = pd.to_numeric(df5[col], errors='coerce')

# Adicionar colunas de ano e fase
df5['ano']=2005
df5['fase']=1

# Mostrar o dataframe resultante
df5

Index(['Código Estab.', 'Código Curso', 'Nome do Estabelecimento',
       'Nome do Curso', 'Grau', 'Vagas Iniciais', 'Colocados ',
       'Colocados (vaga adic.)', 'Nota do últ. colocado (cont. geral)',
       'Sobras para\n2ª fase'],
      dtype='object', name=3)


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0110,Universidade dos Açores - Angra do Heroísmo,0347,Engenharia Zootécnica,10,1,NaN,9,2005,1
5,0110,Universidade dos Açores - Angra do Heroísmo,0625,Educação de Infância,20,9,164.500000,11,2005,1
6,0110,Universidade dos Açores - Angra do Heroísmo,1131,Ciências da Nutrição (Preparatórios),15,15,131.699997,0,2005,1
7,0110,Universidade dos Açores - Angra do Heroísmo,1221,Biotecnologia,10,5,120.599998,5,2005,1
8,0110,Universidade dos Açores - Angra do Heroísmo,1500,Gestão e Conservação da Natureza,15,8,141.199997,7,2005,1
...,...,...,...,...,...,...,...,...,...,...
1061,7230,Escola Superior de Tecnologia da Saúde do Porto,1699,Radiologia,45,45,147.500000,0,2005,1
1062,7230,Escola Superior de Tecnologia da Saúde do Porto,1701,Radioterapia,15,15,148.000000,0,2005,1
1063,7230,Escola Superior de Tecnologia da Saúde do Porto,1728,Saúde Ambiental,45,45,140.800003,0,2005,1
1064,7230,Escola Superior de Tecnologia da Saúde do Porto,1775,Terapêutica da Fala,30,30,156.300003,0,2005,1


In [65]:
# Ler os dados do ficheiro Excel para 2004
df4=pd.read_excel(corrected_urls[20])

# Definir a linha 3 como cabeçalho da tabela e começar a ler os dados a partir da linha 4
df4.columns = df4.iloc[3]
df4 = df4.iloc[4:]

# Remover as últimas 2 linhas que contêm informações irrelevantes
df4 = df4.iloc[:-2]
# Remover a primeira coluna que não é necessária
df4 = df4.iloc[:, 1:]

# Definir o mapeamento dos nomes das colunas originais para os nomes padronizados
column_mapping = {
    'Código Estab.': 'codigo_instituicao',
    'Nome do Estabelecimento': 'instituicao',
    'Código Curso': 'codigo_curso', 
    'Nome do Curso': 'curso',
    'Vagas Iniciais': 'vagas_iniciais',
    'Colocados ': 'colocados',
    'Nota do últ. colocado (cont. geral)': 'nota_ultimo_colocado',
    'Sobras para\n2ª fase': 'vagas_sobrantes'
}

# Mostrar os nomes das colunas para verificação
print(df4.columns)

# Selecionar apenas as colunas necessárias e renomeá-las usando o mapeamento
df4 = df4[column_mapping.keys()]
df4 = df4.rename(columns=column_mapping)

# Converter as colunas numéricas para o tipo correto
numeric_cols = ['vagas_iniciais', 'colocados', 'nota_ultimo_colocado', 'vagas_sobrantes']
for col in numeric_cols:
    if col in df4.columns:
        df4[col] = pd.to_numeric(df4[col], errors='coerce')

# Adicionar colunas de ano e fase
df4['ano']=2004
df4['fase']=1

# Mostrar o dataframe resultante
df4

Index(['Código Estab.', 'Código Curso', 'Nome do Estabelecimento',
       'Nome do Curso', 'Grau', 'Vagas Iniciais', 'Colocados ',
       'Colocados (vaga adic.)', 'Nota do últ. colocado (cont. geral)',
       'Sobras para\n2ª fase'],
      dtype='object', name=3)


3,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase
4,0110,Universidade dos Açores - Angra do Heroísmo,0213,Engenharia do Ambiente,10,1,NaN,9,2004,1
5,0110,Universidade dos Açores - Angra do Heroísmo,0625,Educação de Infância,30,15,121.3,15,2004,1
6,0110,Universidade dos Açores - Angra do Heroísmo,1039,Biotecnologia Agrícola,10,6,114.4,4,2004,1
7,0110,Universidade dos Açores - Angra do Heroísmo,1056,Agricultura Ecológica,10,2,NaN,8,2004,1
8,0110,Universidade dos Açores - Angra do Heroísmo,1131,Ciências da Nutrição (Preparatórios),15,9,127.8,6,2004,1
...,...,...,...,...,...,...,...,...,...,...
1045,7230,Escola Superior de Tecnologia da Saúde do Porto,1699,Radiologia,35,35,144.3,0,2004,1
1046,7230,Escola Superior de Tecnologia da Saúde do Porto,1701,Radioterapia,14,14,144.3,0,2004,1
1047,7230,Escola Superior de Tecnologia da Saúde do Porto,1728,Saúde Ambiental,29,29,131.5,0,2004,1
1048,7230,Escola Superior de Tecnologia da Saúde do Porto,1775,Terapêutica da Fala,20,20,162.0,0,2004,1


In [66]:
dfs=[df24, df23, df22, df21, df20, df19, df18, df17, df16, df15, df14, df13, df12, df11, df10, df9, df8, df7, df6, df5, df4]

df_final = pd.concat(dfs)

# Create derived variables
df_final['taxa_ocupacao'] = df_final['colocados'] / df_final['vagas_iniciais']
df_final['taxa_ocupacao'] = df_final['taxa_ocupacao'].clip(0, 1)

df_final['nome_universidade'] = df_final['instituicao'].str.split(' - ').str[0]
df_final['nome_faculdade'] = df_final['instituicao'].str.split(' - ').str[1]

# Identificador único do curso
df_final['course_id'] = df_final['codigo_curso'].astype(str) + '_' + df_final['codigo_instituicao'].astype(str)

df_final

,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase,taxa_ocupacao,nome_universidade,nome_faculdade,course_id
4,0140,Universidade dos Açores - Faculdade de Ciência...,8031,Ciências Farmacêuticas (Preparatórios),13.0,2.0,152.5,11.0,2024,1,0.153846,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,8031_0140
5,0140,Universidade dos Açores - Faculdade de Ciência...,8086,Medicina Veterinária (Preparatórios),21.0,21.0,164.1,0.0,2024,1,1.000000,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,8086_0140
6,0140,Universidade dos Açores - Faculdade de Ciência...,9022,Ciências Agrárias,16.0,5.0,NaN,11.0,2024,1,0.312500,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022_0140
7,0140,Universidade dos Açores - Faculdade de Ciência...,L344,Guias de Natureza e Património,19.0,7.0,120.3,12.0,2024,1,0.368421,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,L344_0140
8,0150,Universidade dos Açores - Faculdade de Ciência...,9135,Estudos Europeus,20.0,17.0,115.6,3.0,2024,1,0.850000,Universidade dos Açores,Faculdade de Ciências Sociais e Humanas,9135_0150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,7230,Escola Superior de Tecnologia da Saúde do Porto,1699,Radiologia,35.0,35.0,144.3,0.0,2004,1,1.000000,Escola Superior de Tecnologia da Saúde do Porto,NaN,1699_7230
1046,7230,Escola Superior de Tecnologia da Saúde do Porto,1701,Radioterapia,14.0,14.0,144.3,0.0,2004,1,1.000000,Escola Superior de Tecnologia da Saúde do Porto,NaN,1701_7230
1047,7230,Escola Superior de Tecnologia da Saúde do Porto,1728,Saúde Ambiental,29.0,29.0,131.5,0.0,2004,1,1.000000,Escola Superior de Tecnologia da Saúde do Porto,NaN,1728_7230
1048,7230,Escola Superior de Tecnologia da Saúde do Porto,1775,Terapêutica da Fala,20.0,20.0,162.0,0.0,2004,1,1.000000,Escola Superior de Tecnologia da Saúde do Porto,NaN,1775_7230


In [67]:
## Data Cleaning
df_clean=df_final.copy()

In [68]:
# Mostrar as dimensões (linhas x colunas) do dataset
print("Dimensões do dataset:", df_clean.shape)

# Listar todas as colunas disponíveis no dataset
print("\nColunas disponíveis:")
for i, col in enumerate(df_clean.columns):
    print(f"{i+1:2d}. {col}")

# Mostrar estatísticas básicas sobre o dataset
print(f"\nPeríodo temporal: {df_clean['ano'].min()} - {df_clean['ano'].max()}")
print(f"Número de instituições: {df_clean['codigo_instituicao'].nunique()}")
print(f"Número de cursos: {df_clean['codigo_curso'].nunique()}")

Dimensões do dataset: (22621, 14)

Colunas disponíveis:
 1. codigo_instituicao
 2. instituicao
 3. codigo_curso
 4. curso
 5. vagas_iniciais
 6. colocados
 7. nota_ultimo_colocado
 8. vagas_sobrantes
 9. ano
10. fase
11. taxa_ocupacao
12. nome_universidade
13. nome_faculdade
14. course_id

Período temporal: 2004 - 2024
Número de instituições: 220
Número de cursos: 1538


In [69]:
# Verificar valores anómalos nos dados:

# 1. Notas superiores a 200 (impossível no sistema português de classificação 0-200)
df_clean[df_clean['nota_ultimo_colocado'] > 200]

# 2. Taxa de ocupação superior a 1 (impossível ter mais colocados que vagas)
df_clean[df_clean['taxa_ocupacao'] > 1]

# 3. Número negativo de vagas (impossível ter vagas negativas)
df_clean[df_clean['vagas_iniciais'] < 0]

# Find rows where colocados > vagas_iniciais
mask = df_clean['colocados'] > df_clean['vagas_iniciais']

# Set vagas_iniciais equal to colocados where mask is True
df_clean.loc[mask, 'vagas_iniciais'] = df_clean.loc[mask, 'colocados']
# Display the rows that were fixed
df_clean[mask]
###corrigir estes (acertar vagas iniciais com numero de colocados)

,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase,taxa_ocupacao,nome_universidade,nome_faculdade,course_id
10,0150,Universidade dos Açores - Faculdade de Ciência...,9219,Psicologia,38.0,38.0,147.900000,0.0,2021,1,1.0,Universidade dos Açores,Faculdade de Ciências Sociais e Humanas,9219_0150
15,0150,Universidade dos Açores - Faculdade de Ciência...,L041,Estudos Portugueses e Ingleses,25.0,25.0,135.800000,0.0,2021,1,1.0,Universidade dos Açores,Faculdade de Ciências Sociais e Humanas,L041_0150
25,0201,Universidade do Algarve - Faculdade de Ciência...,8509,Património Cultural e Arqueologia,36.0,36.0,109.700000,0.0,2021,1,1.0,Universidade do Algarve,Faculdade de Ciências Humanas e Sociais,8509_0201
26,0201,Universidade do Algarve - Faculdade de Ciência...,9204,"Línguas, Literaturas e Culturas",40.0,40.0,113.300000,0.0,2021,1,1.0,Universidade do Algarve,Faculdade de Ciências Humanas e Sociais,9204_0201
27,0201,Universidade do Algarve - Faculdade de Ciência...,9219,Psicologia,69.0,69.0,158.800000,0.0,2021,1,1.0,Universidade do Algarve,Faculdade de Ciências Humanas e Sociais,9219_0201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402,7003,Escola Superior de Enfermagem do Porto,1169,Enfermagem,271.0,271.0,168.500000,0.0,2007,1,1.0,Escola Superior de Enfermagem do Porto,NaN,1169_7003
1416,7020,Instituto Politécnico de Castelo Branco - Esco...,1023,Análises Clínicas e de Saúde Pública,26.0,26.0,153.000000,0.0,2007,1,1.0,Instituto Politécnico de Castelo Branco,Escola Superior de Saúde Dr. Lopes Dias,1023_7020
1437,7045,Instituto Politécnico de Leiria - Escola Super...,9500,Enfermagem,61.0,61.0,150.800003,0.0,2007,1,1.0,Instituto Politécnico de Leiria,Escola Superior de Saúde de Leiria,9500_7045
1438,7045,Instituto Politécnico de Leiria - Escola Super...,9501,Enfermagem (entrada no 2.º semestre),62.0,62.0,138.399994,0.0,2007,1,1.0,Instituto Politécnico de Leiria,Escola Superior de Saúde de Leiria,9501_7045


In [70]:
# Verificar valores em falta em cada coluna
print("Valores em falta por coluna:")
missing_data = df_clean.isnull().sum()
print(missing_data[missing_data > 0])

Valores em falta por coluna:
nota_ultimo_colocado    1097
nome_faculdade          3736
dtype: int64


In [71]:
## Input missing data (nota do ultimo colocado)

# Ordenar por ano e curso para garantir que os valores do ano anterior estão disponíveis
df_clean = df_clean.sort_values(['ano', 'course_id'])

# Preencher valores NaN de colocados com o valor do ano anterior usando shift
df_clean['nota_ultimo_colocado'] = df_clean.groupby('course_id')['nota_ultimo_colocado'].transform(lambda x: x.fillna(x.shift(1)))

# Verificar se ainda existem valores em falta
print("Valores em falta por coluna:")
missing_data = df_clean.isnull().sum()
print(missing_data[missing_data > 0])

Valores em falta por coluna:
nota_ultimo_colocado     679
nome_faculdade          3736
dtype: int64


In [72]:
df_clean['curso'] = df_clean['curso'].str.strip()

df_clean 

,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase,taxa_ocupacao,nome_universidade,nome_faculdade,course_id
62,0300,Universidade de Aveiro,0003,Administração Pública,35.0,35.0,133.0,0.0,2004,1,1.000000,Universidade de Aveiro,NaN,0003_0300
151,0502,Universidade de Coimbra - Faculdade de Direito,0003,Administração Pública,50.0,30.0,116.0,20.0,2004,1,0.600000,Universidade de Coimbra,Faculdade de Direito,0003_0502
360,1000,Universidade do Minho,0003,Administração Pública,45.0,41.0,107.4,4.0,2004,1,0.911111,Universidade do Minho,NaN,0003_1000
129,0501,Universidade de Coimbra - Faculdade de Ciência...,0016,Antropologia,45.0,45.0,112.0,0.0,2004,1,1.000000,Universidade de Coimbra,Faculdade de Ciências e Tecnologia,0016_0501
282,0804,Universidade Técnica de Lisboa - Instituto Sup...,0016,Antropologia,49.0,36.0,104.5,13.0,2004,1,0.734694,Universidade Técnica de Lisboa,Instituto Superior de Ciências Sociais e Polít...,0016_0804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0400,Universidade da Beira Interior,L331,Computação Criativa e Realidade Virtual,20.0,19.0,126.8,1.0,2024,1,0.950000,Universidade da Beira Interior,NaN,L331_0400
725,3092,Instituto Politécnico da Guarda - Escola Super...,L335,Design de Equipamento e Ambientes,24.0,19.0,117.9,5.0,2024,1,0.791667,Instituto Politécnico da Guarda,Escola Superior de Tecnologia e Gestão,L335_3092
7,0140,Universidade dos Açores - Faculdade de Ciência...,L344,Guias de Natureza e Património,19.0,7.0,120.3,12.0,2024,1,0.368421,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,L344_0140
390,1203,Universidade de Trás-os-Montes e Alto Douro - ...,L345,Cidades Sustentáveis e Inteligentes,25.0,2.0,113.5,23.0,2024,1,0.080000,Universidade de Trás-os-Montes e Alto Douro,Escola de Ciências e Tecnologia,L345_1203


In [73]:
# Remover linhas que ainda têm valores NaN
df_clean.dropna(inplace=True)

In [74]:
df_clean.loc[df_clean['course_id']=='9011_1503']

,codigo_instituicao,instituicao,codigo_curso,curso,vagas_iniciais,colocados,nota_ultimo_colocado,vagas_sobrantes,ano,fase,taxa_ocupacao,nome_universidade,nome_faculdade,course_id
400,1503,Universidade de Lisboa - Faculdade de Ciências,9011,Biologia,181.0,181.0,145.0,0.0,2014,1,1.0,Universidade de Lisboa,Faculdade de Ciências,9011_1503
399,1503,Universidade de Lisboa - Faculdade de Ciências,9011,Biologia,180.0,180.0,144.8,0.0,2015,1,1.0,Universidade de Lisboa,Faculdade de Ciências,9011_1503
398,1503,Universidade de Lisboa - Faculdade de Ciências,9011,Biologia,180.0,180.0,139.5,0.0,2016,1,1.0,Universidade de Lisboa,Faculdade de Ciências,9011_1503
402,1503,Universidade de Lisboa - Faculdade de Ciências,9011,Biologia,180.0,180.0,146.0,0.0,2017,1,1.0,Universidade de Lisboa,Faculdade de Ciências,9011_1503
404,1503,Universidade de Lisboa - Faculdade de Ciências,9011,Biologia,174.0,174.0,147.5,0.0,2018,1,1.0,Universidade de Lisboa,Faculdade de Ciências,9011_1503
403,1503,Universidade de Lisboa - Faculdade de Ciências,9011,Biologia,174.0,174.0,147.5,0.0,2019,1,1.0,Universidade de Lisboa,Faculdade de Ciências,9011_1503
404,1503,Universidade de Lisboa - Faculdade de Ciências,9011,Biologia,195.0,195.0,160.8,0.0,2020,1,1.0,Universidade de Lisboa,Faculdade de Ciências,9011_1503
407,1503,Universidade de Lisboa - Faculdade de Ciências,9011,Biologia,175.0,175.0,169.5,0.0,2021,1,1.0,Universidade de Lisboa,Faculdade de Ciências,9011_1503
420,1503,Universidade de Lisboa - Faculdade de Ciências,9011,Biologia,174.0,174.0,167.0,0.0,2022,1,1.0,Universidade de Lisboa,Faculdade de Ciências,9011_1503
429,1503,Universidade de Lisboa - Faculdade de Ciências,9011,Biologia,174.0,174.0,160.5,0.0,2023,1,1.0,Universidade de Lisboa,Faculdade de Ciências,9011_1503


In [75]:
df_clean.to_csv('cleaned_data.csv', index=False)